# ESPN API Endpoint Exploration

This notebook fetches data from various ESPN API endpoints as defined in `docs/api_endpoint_inventory.md`.

**Instructions:**
1. Fill in the `sample_ids` dictionary in the second code cell with valid IDs for parameters like `season_id`, `event_id`, etc., to test specific endpoints.
2. Run the cells sequentially.
3. JSON responses will be saved in the `sample_responses/` directory within your project root.

In [5]:
import json
import re
from datetime import datetime
from pathlib import Path

import requests

In [6]:
BASE_URL = "http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball"
OUTPUT_DIR = Path("../docs/sample_responses")

# TODO: Fill in these sample IDs to test endpoints that require them.
# If an ID is not provided for a required parameter, the endpoint will be skipped.
sample_ids = {
    "athlete_id": "4066764",  # TODO: Fill in a sample ID
    "team_id": "2261",  # TODO: Fill in a sample ID
    "event_id": "401582150",  # TODO: Fill in a sample ID
    "season_id": "2024",  # TODO: Fill in a sample ID
    "split_id": "0",  # TODO: Fill in a sample ID
    "type_id": "2",  # TODO: Fill in a sample ID
    "venue_id": "2225",  # TODO: Fill in a sample ID
    "week_id": "3",  # TODO: Fill in a sample ID
}


def slugify(text):
    """Convert text to a filesystem-safe slug."""
    text = text.lower()
    text = re.sub(r"[^\w\s-]", "", text)  # Remove special characters
    text = re.sub(r"[\s_-]+", "-", text)  # Replace whitespace/underscores with hyphens
    text = text.strip("-")
    return text

In [7]:
endpoints_to_process = [
    {"title": "List Seasons", "path_template": "/seasons", "required_params": []},
    {
        "title": "Season Details",
        "path_template": "/seasons/{season_id}",
        "required_params": ["season_id"],
    },
    {
        "title": "List Season Types",
        "path_template": "/seasons/{season_id}/types",
        "required_params": ["season_id"],
    },
    {
        "title": "Season Type Details",
        "path_template": "/seasons/{season_id}/types/{type_id}",
        "required_params": ["season_id", "type_id"],
    },
    {
        "title": "List Weeks",
        "path_template": "/seasons/{season_id}/types/{type_id}/weeks",
        "required_params": ["season_id", "type_id"],
    },
    {
        "title": "Week Details",
        "path_template": "/seasons/{season_id}/types/{type_id}/weeks/{week_id}",
        "required_params": ["season_id", "type_id", "week_id"],
    },
    {
        "title": "List Week Events",
        "path_template": "/seasons/{season_id}/types/{type_id}/weeks/{week_id}/events",
        "required_params": ["season_id", "type_id", "week_id"],
    },
    {
        "title": "Event Details",
        "path_template": "/events/{event_id}",
        "required_params": ["event_id"],
    },
    {
        "title": "Season Specific Team Details",
        "path_template": "/seasons/{season_id}/teams/{team_id}",
        "required_params": ["season_id", "team_id"],
    },
    {
        "title": "Score Details",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/score",
        "required_params": ["event_id", "team_id"],
    },
    {
        "title": "Linescores",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/linescores",
        "required_params": ["event_id", "team_id"],
    },
    {
        "title": "Statistics",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/statistics",
        "required_params": ["event_id", "team_id"],
    },
    {
        "title": "Player Statistics (per Game)",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/roster/{athlete_id}/statistics/{split_id}",
        "required_params": ["event_id", "team_id", "athlete_id", "split_id"],
    },
    {
        "title": "Leaders",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/leaders",
        "required_params": ["event_id", "team_id"],
    },
    {
        "title": "Roster",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/roster",
        "required_params": ["event_id", "team_id"],
    },
    {
        "title": "Record",
        "path_template": "/events/{event_id}/competitions/{event_id}/competitors/{team_id}/records",
        "required_params": ["event_id", "team_id"],
    },
    {
        "title": "Venue Details",
        "path_template": "/venues/{venue_id}",
        "required_params": ["venue_id"],
    },
    {
        "title": "Situation",
        "path_template": "/events/{event_id}/competitions/{event_id}/situation",
        "required_params": ["event_id"],
    },
    {
        "title": "Status",
        "path_template": "/events/{event_id}/competitions/{event_id}/status",
        "required_params": ["event_id"],
    },
    {
        "title": "Odds",
        "path_template": "/events/{event_id}/competitions/{event_id}/odds",
        "required_params": ["event_id"],
    },
    {
        "title": "Play-by-Play",
        "path_template": "/events/{event_id}/competitions/{event_id}/plays",
        "required_params": ["event_id"],
    },
]

In [8]:
# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base URL: {BASE_URL}")
print(f"Sample responses will be saved to: {OUTPUT_DIR.resolve()}")
print("\nProcessing endpoints...\n")

for endpoint_info in endpoints_to_process:
    title = endpoint_info["title"]
    path_template = endpoint_info["path_template"]
    required_params = endpoint_info["required_params"]

    print(f"--- Processing: {title} ({path_template}) ---")

    # Check if all required params have sample IDs
    params_to_fill = {}
    can_process = True
    for param_name in required_params:
        if sample_ids.get(param_name):
            params_to_fill[param_name] = sample_ids[param_name]
        else:
            print(
                f"Skipping endpoint: Missing sample ID for required parameter '{param_name}'. Please define it in 'sample_ids'."
            )
            can_process = False
            break

    if not can_process:
        print("\n")
        continue

    # Format the path if it has placeholders
    try:
        current_path = path_template.format(**params_to_fill)
    except KeyError as e:
        print(
            f"Skipping endpoint: Error formatting path. Missing key {e} in 'sample_ids' for path '{path_template}'."
        )
        print("\n")
        continue

    full_url = BASE_URL + current_path

    print(f"Requesting URL: {full_url}")

    try:
        response = requests.get(full_url, timeout=30)
        response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)

        response_data = response.json()

        # Generate filename
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        slug_title = slugify(title)
        filename = f"{slug_title}_response_{timestamp}.json"
        filepath = OUTPUT_DIR / filename

        with open(filepath, "w") as f:
            json.dump(response_data, f, indent=4)
        print(f"Successfully fetched and saved response to: {filepath}")

    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code} for URL: {full_url}")
        print(f"Response content: {e.response.text[:500]}...")  # Print first 500 chars of error
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except json.JSONDecodeError:
        print(f"Failed to decode JSON response from URL: {full_url}")
        print(f"Response content: {response.text[:500]}...")

    print("\n")

print("--- Finished processing all defined endpoints ---")

Base URL: http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball
Sample responses will be saved to: /Users/tmcdonne/projects/ncaa_basketball_pipeline/docs/sample_responses

Processing endpoints...

--- Processing: List Seasons (/seasons) ---
Requesting URL: http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons
Successfully fetched and saved response to: ../docs/sample_responses/list-seasons_response_20250507103536.json


--- Processing: Season Details (/seasons/{season_id}) ---
Requesting URL: http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024
Successfully fetched and saved response to: ../docs/sample_responses/season-details_response_20250507103536.json


--- Processing: List Season Types (/seasons/{season_id}/types) ---
Requesting URL: http://sports.core.api.espn.com/v2/sports/basketball/leagues/mens-college-basketball/seasons/2024/types
Successfully fetched and saved r

## Next Steps

- Review the saved JSON responses in the `sample_responses/` directory.
- Use these samples for developing and testing the `dlt` pipeline resources.